In [1]:
#@title Carregar bibliotecas

# Libraries

import requests # método request
import os # comandos de arquivo/diretório
import pandas as pd # pandas dataframe
import numpy as np # numpy
import sys
import shutil # apagar uma árvore de pastas
import zipfile
import io
from bs4 import BeautifulSoup
import re

print("Bibliotecas carregadas.")

Bibliotecas carregadas.


In [ ]:
# Overriding the default warning filter to hide ssl warnings from users
# Comment the lines bellow when running tests or some not working
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
# Classes e funções

# Classes para acessar as informações da aplicativo web do INPI
class api_inpi:
    def __init__(self, url):
        self.url = url
    
    # Verifica o código de status do arquivo no servidor
    def url_status(self):
        r = requests.head(self.url)
        return r.status_code
    
    # Faz o download do arquivo no pacote zip com a extensão escolhida para a memória
    def xml_buffer(self, ext):
        if self.url_status() == 200: # Se o código for 200, procede com download
            r = requests.get(self.url) # Primeiro faz a requisição do arquivo para o servidor        
            f_zip = zipfile.ZipFile(io.BytesIO(r.content)) # para simular um objeto do tipo bytes para o arquivo zip
            f_names = f_zip.namelist() # depois lista todos os arquivos do pacote compactado
            f_name = [i for i in f_names if ext in i][0] # para localizar o arquivo com a extensão desejada
            return f_zip.read(f_name) # retornando o conteúdo do arquivo para a memória
        if self.url_status() == 302: # Mas se o código for 302, o arquivo ainda não existe
            print("404: O arquivo solicitado não existe.") # então imprime uma mensagem de erro
            
    # Faz a busca de patentes com até três critérios para o titular
    def busca_titular(self,revista,org,df):
        rpi = revista # número da RPI a ser verificada
        ext = ".xml" # extensão do arquivo para ser extraído do pacote zip
        xml_data = self.xml_buffer(ext) # verifica o status do arquivo no servidor e realiza o download para a memória
        xml_data = re.sub(b' +',b' ',xml_data) # substitui espaçamentos múltiplos por espaçamentos únicos
        soup = BeautifulSoup(xml_data, "xml") # passing the stored data inside the beautifulsoup parser, storing the returned object
        
        org_soup = soup.find_all("nome-completo", text=lambda x: org in x.casefold()) # Busca quantas vezes o nome da(s) organização(ões) apareceu na RPI
                
        despacho_len = len(org_soup) # quantidade total de registros com os critérios solicitados
        for i in range(despacho_len): # percorre o arquivo da revista em busca dos despachos com os critérios solicitados
            despacho_soup = org_soup[i].find_parent('despacho')
            
            # Procura o código de despacho
            cod_despacho = despacho_soup.find('codigo')
            if cod_despacho != None:
                cod_despacho = cod_despacho.text
        
            # Procura o título da invenção
            titulo = despacho_soup.find("titulo", inid="54")
            if titulo != None:
                titulo = titulo.text
        
            # Procura o número da proteção
            registro = despacho_soup.find('numero')
            if registro != None:
                registro = registro.text
                
            # Monta o codigo de busca para o servidor
            cod_busca = registro[:-2].replace(' ', '')
            #gestao_df['busca'] = gestao_df['registro'] # Cria a coluna busca com base na PI
            #gestao_df['busca'] = gestao_df['busca'].str[:-2] # remove o digito identificador
            #gestao_df['busca'] = gestao_df['busca'].str.replace(' ', '') # remove os espaços em branco

            # Procura o kind code
            kindcode = despacho_soup.find('numero').attrs.get('kindcode')
    
            # Procura a data de depósito
            data_deposito = despacho_soup.find('data-deposito')
            if data_deposito != None:
                data_deposito = data_deposito.text

            # Procura os dados de classificação internacional
            ci_len = len(despacho_soup.find_all("classificacao-internacional"))
            ci = ""
            if ci_len > 0:
                for i in range(ci_len):
                    ci_ano = despacho_soup.find_all("classificacao-internacional")[i].text + " (" + despacho_soup.find_all("classificacao-internacional")[i].attrs.get('ano') + ")"
                    ci = ci + ci_ano + " | "
            ci = ci[:-3]
            
            # Verifica se é gestão interna
            gestao = "Sim"
            titular_1 = despacho_soup.find("titular", sequencia="1").find("nome-completo").text
            if org.lower() not in titular_1.lower(): # Caso o primeiro titular não seja a organização procurada, a gestão é externa
                gestao = "Não"

            # Procura a lista de titulares
            titular_len = len(despacho_soup.find_all("titular"))
            titulares = ""
            if titular_len > 0:
                for i in range(titular_len):
                    titular = despacho_soup.find_all("titular")[i].find("nome-completo").text
                    titulares = titulares + titular + " | "
            titulares = titulares[:-3]
    
            # Procura a lista de inventores
            inventor_len = len(despacho_soup.find_all("inventor"))
            inventores = ""
            if inventor_len > 0:
                for i in range(inventor_len):
                    inventor = despacho_soup.find_all("inventor")[i].find("nome-completo").text
                    inventores = inventores + inventor + " | "
            inventores = inventores[:-3]

            df.loc[len(df)] = [rpi, registro, kindcode, cod_despacho, gestao, titulares, inventores, data_deposito, ci, titulo, cod_busca]
            #df = df.append(rpi, registro, kindcode, cod_despacho, gestao, titulares, inventores, data_deposito, ci, titulo)
    
def excel_report(xls_df,xls_name,sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    # also set the default datetime and date formats.
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy')
    # Convert the dataframe to an XlsxWriter Excel object.
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the xlsxwriter workbook and worksheet objects in order to set the column
    # widths, to make the dates clearer.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    # Close the Pandas Excel writer and output the Excel file
    writer.save()
    print("Relatório concluído.")
        
def gestao_report(xls_df,xls_name,sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    # also set the default datetime and date formats.
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy')
    # Convert the dataframe to an XlsxWriter Excel object.
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the xlsxwriter workbook and worksheet objects in order to set the column
    # widths, to make the dates clearer.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    # Add format styles to the the lines that need download
    need_download = workbook.add_format()
    need_download.set_bg_color('yellow')
    need_download.set_font_color('red')
    # Set the columns lenghts
    worksheet.set_column('E:E', 80)
    # Close the Pandas Excel writer and output the Excel file
    writer.save()
    print("Relatório concluído.")
        
def parecer_report(xls_df,xls_name,sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    # also set the default datetime and date formats.
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy')
    # Convert the dataframe to an XlsxWriter Excel object.
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the xlsxwriter workbook and worksheet objects in order to set the column
    # widths, to make the dates clearer.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    # Add format styles to the the lines that need download
    need_download = workbook.add_format()
    need_download.set_bg_color('yellow')
    need_download.set_font_color('red')
    # Set the columns lenghts
    worksheet.set_column('A:A', 20)
    worksheet.set_column('B:B', 20)
    worksheet.set_column('C:C', 10)
    worksheet.set_column('D:D', 30)
    worksheet.set_column('E:E', 60)
    worksheet.set_column('F:F', 50)
    # Set conditional format to 16.1
    worksheet.conditional_format('F2:F50', {'type': 'cell',
                                        'criteria': 'equal to',
                                        'value': '"Faça o donwload da carta patente no site INPI."',
                                        'format': need_download})
    # Set conditional format to error cases
    worksheet.conditional_format('F2:F50', {'type': 'cell',
                                        'criteria': 'equal to',
                                        'value': '"Erro! Verifique o site do INPI!"',
                                        'format': need_download})
    # Ser conditional format to duplicate files
    worksheet.conditional_format('F2:F50', {'type': 'cell',
                                        'criteria': 'equal to',
                                        'value': '"Arquivos duplicados, verifique o site do INPI."',
                                        'format': need_download})
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print("Relatório concluído.")
    
print("Classes e funções definidas.")

Classes e funções definidas.


In [5]:
# Busca os pareceres para uma instituição
# Usar: api_inpi(url).busca_titular(rpi,"nome",dataframe)
# url = url da seção de patentes da rpi para a revista escolhida (str)
# rpi = numero da revista (int)
# nome = nome completo da instituição em lower case (str)
# dataframe = nome do dataframe para armazenamento

rpi = 2587
url = "http://revistas.inpi.gov.br/txt/P" + str(rpi) + ".zip"
column_names = ['rpi', 'registro', 'kind code', 'cod_despacho', 'gestão', 'titulares', 'inventores', 'data de deposito', 'classificação internacional', 'título', 'cod_busca'] # cria as colunas do dataframe
gestao_df = pd.DataFrame(columns=column_names) # cria o dataframe

api_inpi(url).busca_titular(rpi,"universidade estadual de campinas",gestao_df)

gestao_df

,rpi,registro,kind code,cod_despacho,gestão,titulares,inventores,data de deposito,classificação internacional,título,cod_busca
0,2587,BR 10 2020 003684-0,None,2.1,Não,CNPEM - CENTRO NACIONAL DE PESQUISA EM ENERGIA...,,21/02/2020,,None,BR102020003684
1,2587,BR 10 2020 015176-2,None,2.10,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/07/2020,,None,BR102020015176
2,2587,BR 10 2015 024497-5,A2,6.1,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/09/2015,,None,BR102015024497
3,2587,BR 13 2013 023109-5,E2,6.1,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,10/09/2013,,None,BR132013023109
4,2587,PI 1010503-4,A2,6.1,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/09/2010,,None,PI1010503
5,2587,PI 1105298-8,A2,6.1,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,22/12/2011,,None,PI1105298
6,2587,BR 10 2016 028096-6,A2,6.21,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP | ...,,28/11/2016,,None,BR102016028096
7,2587,BR 10 2013 001653-5,A2,6.22,Não,Universidade Federal de Juiz de Fora | Univers...,,23/01/2013,,None,BR102013001653
8,2587,PI 1012502-7,A2,6.22,Sim,Universidade estadual de Campinas - UNICAMP,,24/11/2010,,None,PI1012502
9,2587,BR 10 2013 033416-2,A2,7.7,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,19/12/2013,,None,BR102013033416


In [22]:
# Mescla a coluna extraída da RPI com a do SICI

arquivo = 'download' # nome do arquivo
arquivo_df = pd.read_html(arquivo, encoding='utf-8', header=0)[0].iloc[:-1] # Converte o arquivo html em um dataframe e remove a última linha
arquivo_df.columns = map(str.lower, arquivo_df.columns) # coloca o nome das colunas em minúsculo
arquivo_df.drop(arquivo_df.columns.difference(['pi', 'tecnologia', 'gesta o']), axis=1, inplace=True)
arquivo_df.columns = ['apelido','pi','gestão sici']

# Cria a coluna busca no dataframe
arquivo_df['busca'] = arquivo_df['pi'] # Cria a coluna busca com base na PI
arquivo_df['busca'] = arquivo_df['busca'].str[:-2] # remove o digito identificador
arquivo_df['busca'] = arquivo_df['busca'].str.replace(' ', '') # remove os espaços em branco

rpi_df = arquivo_df.merge(gestao_df, how='outer', indicator=True) # mescla os dataframe sici e inpi

rpi_df = rpi_df[['rpi', 'apelido','registro', 'kind code','cod_despacho','gestão sici','gestão','titulares', 'inventores', 'data de deposito', 'classificação internacional', 'título']]

rpi_df

# gestao_report(gestao_df,"rpi_sici_gestao_" + str(rpi),str(rpi))

,rpi,apelido,registro,kind code,cod_despacho,gestão sici,gestão,titulares,inventores,data de deposito,classificação internacional,título
0,2587,1528_NANOCELULOSE,BR 10 2020 003684-0,None,2.1,Sim,Não,CNPEM - CENTRO NACIONAL DE PESQUISA EM ENERGIA...,,21/02/2020,,None
1,2587,1513_ELETROSSINTESE OURO,BR 10 2020 015176-2,None,2.10,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/07/2020,,None
2,2587,959_RESTAURADORES,BR 10 2015 024497-5,A2,6.1,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/09/2015,,None
3,2587,751_PLGA,BR 13 2013 023109-5,E2,6.1,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,10/09/2013,,None
4,2587,413_DEPOSICAO,PI 1010503-4,A2,6.1,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,24/09/2010,,None
5,2587,517_INCORPORAÇÃO,PI 1105298-8,A2,6.1,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,22/12/2011,,None
6,2587,1107_BIOCATALISE,BR 10 2016 028096-6,A2,6.21,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP | ...,,28/11/2016,,None
7,2587,644_ECODOR,BR 10 2013 001653-5,A2,6.22,Sim,Não,Universidade Federal de Juiz de Fora | Univers...,,23/01/2013,,None
8,2587,437_DESCELULARIZACAO,PI 1012502-7,A2,6.22,Sim,Sim,Universidade estadual de Campinas - UNICAMP,,24/11/2010,,None
9,2587,750_JABUTICABA,BR 10 2013 033416-2,A2,7.7,Sim,Sim,UNIVERSIDADE ESTADUAL DE CAMPINAS - UNICAMP,,19/12/2013,,None


In [11]:
#@title Carregar arquivo do SICI

# # Para uso no Google Colab

# from google.colab import files # para uso no google colab
# !pip install -q XlsxWriter # para uso no google colab

# print('\033[1m' + "\nCarregue o arquivo do SICI com a revista da semana\n" + '\033[0m')

# uploaded = files.upload()

# for fn in uploaded.keys():
#   arquivo=fn

In [3]:
#@title Fazer o download dos arquivos

# Comentar essa linha se for usar no Google Colab
arquivo = 'download' # nome do arquivo

sici_df = pd.read_html(arquivo, encoding='utf-8', header=0)[0].iloc[:-1] # Converte o arquivo html em um dataframe e remove a última linha
revista = sici_df.iloc[0]['Revista'] # extrai o número da revista da primeira linha
sici_df.columns = map(str.lower, sici_df.columns) # coloca o nome das colunas em minúsculo

# Cria a coluna com o número da tecnologia
sici_df[['nr_sici', 'sici_name']] = sici_df['tecnologia'].str.split('_', 1, expand=True) #split only in the first pattern finded
sici_df['nr_sici'] = sici_df['nr_sici'].str.zfill(3) # se o número da tecnologia for menor que 100 adiciona zeros na frente
sici_df['nr_sici'] = sici_df['nr_sici'] + "_" # adiciona underline após o número
sici_df.nr_sici.replace('000_', "", inplace=True) # remove os valores 000_ de número que aparecem se for padrão de nome com tecnologia antiga
sici_df.drop(['tecnologia'], axis=1, inplace=True) # remove as colunas tecnologia antiga para criar de novo com o padrão 00X
sici_df['tecnologia'] = sici_df['nr_sici'] + sici_df['sici_name'] # recria a coluna tecnologia com padrão 00X

# Se a tecnologia for antiga (nome começa com _), substitui tecnologia por pi
sici_df.tecnologia.replace("", sici_df.pi, regex=True, inplace=True)

# Procurando tecnologias com dois pareceres diferentes duplicados
sici_df['dupl_exig'] = sici_df['pi'].duplicated(keep=False)

i=0
for tecnologia,dupl_exig,despacho in zip(sici_df['tecnologia'],sici_df['dupl_exig'],sici_df['despacho']):
    if dupl_exig == True:
        sici_df.at[i, 'tecnologia'] = tecnologia + "_" + despacho
    i+=1
    
# Se a tecnologia for antiga (nome começa com _), substitui tecnologia por pi
sici_df.tecnologia.replace('^_.*$', sici_df.pi, regex=True, inplace=True)

# Cria as colunas com os nomes dos arquivos com base nos despachos
sici_df['doc_name'] = "EXIGENCIA " + sici_df['despacho']

print('\033[1m' + "\nRPI %s\n" %(revista) + '\033[0m')

print('\033[1m' + "\nDados do SICI convertidos para um dataframe.\n" + '\033[0m')

###### Download dos arquivos no servidor do INPI

## Verifica o nome dos arquivos no servidor do INPI com base no campo busca

url = 'https://parecer.inpi.gov.br/arquivos/RPI/' + revista

# verifica se os arquivos já estão disponíveis no servidor
if requests.get(url, verify=False).status_code != 200:
    print('\033[1m' + "\nNão existem arquivos no servidor para esta revista.\n" + '\033[0m')
    sys.exit()

print('\033[1m' + "\nOs arquivos estão disponíveis para download. Iniciando o processo.....\n" + '\033[0m')

# arquivos_inpi = requests.get(url, verify=False)
finpi_df = pd.read_html(requests.get(url, verify=False).content, encoding='utf-8', header=0)[0].iloc[2:-1]

## EXTRAIR A DATA DA REVISTA DO DATAFRAME

data_inpi = finpi_df.iloc[0]['Last modified'] # extrai a data da primeira linha
# Monta o dicionário de correspondência dos meses
dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05",
       "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10",
       "Nov": "11", "Dec": "12"}
data = data_inpi[0:2] + dict[data_inpi[3:6]] + data_inpi[9:11] # DDMMAA

print('\033[1m' + "\nData da revista: %s\n" %(data) + '\033[0m')

## Folder e coluna de busca

sici_df['folder'] = sici_df['tecnologia'] + "/" + sici_df['nr_sici'] + "EXIGENCIAS"

print('\033[1m' + "\nEstrutura de pastas montada.\n" + '\033[0m')

# Cria a coluna busca no dataframe
sici_df['busca'] = sici_df['pi'] # Cria a coluna busca com base na PI
sici_df['busca'] = sici_df['busca'].str[:-2] # remove o digito identificador
sici_df['busca'] = sici_df['busca'].str.replace(' ', '') # remove os espaços em branco
sici_df['busca'] = sici_df['busca'].str.replace('BR', '') # remove o código BR

print('\033[1m' + "\nColuna de busca defininda.\n" + '\033[0m')

sici_df.drop(sici_df.columns.difference(['tecnologia', 'pi', 'despacho', 'nr_sici', 'doc_name', 'folder', 'busca', 'dupl_exig']), axis=1, inplace=True)

finpi_df.drop(finpi_df.columns.difference(['Name']), axis=1, inplace=True)
finpi_df.columns = ['inpi_name'] # renomeia as colunas do inpi
finpi_df['busca'] = finpi_df['inpi_name'].str[3:-11] # cria a coluna com o código de busca
finpi_df = finpi_df.reset_index(drop=True)

print('\033[1m' + "\nDataframe do INPI preparado.\n" + '\033[0m')

##### Mesclar os dataframes e organizar os dados

pareceres_df = sici_df.merge(finpi_df, how='outer', indicator=True) # mescla os dataframe sici e inpi

pareceres_df.dropna(subset = ['pi'], inplace=True) # remove as linhas onde pi é NaN

pareceres_df.inpi_name.replace(np.nan, False, inplace=True) # troca NaN por False no inpi_name
pareceres_df.loc[pareceres_df['inpi_name'] != False, 'download'] = True # se existir nome do arquivo, seta downloa como True
pareceres_df.loc[pareceres_df['inpi_name'] == False, 'download'] = False # se não, seta download como False

pareceres_df.drop(['_merge','busca'], axis=1, inplace=True) # remove as colunas _merge e busca

print('\033[1m' + "\nDataframe dos pareceres criado.\n" + '\033[0m')

del [[sici_df,finpi_df]]

print('\033[1m' + "\nDataframes sici_df e finpi_df deletados.\n" + '\033[0m')

###### Adicionando flags de contagem

# Procurando arquivos duplicados
pareceres_df['dupl'] = pareceres_df['pi'].duplicated(keep=False)
pareceres_df['flag'] = pareceres_df.groupby('pi').cumcount()
pareceres_df.loc[pareceres_df['download'] == False, 'flag'] = False # flag é false se não houver download
pareceres_df.loc[pareceres_df['dupl'] == False, 'flag'] = "" # flag é vazio se não houver duplicidade

# Muda a numeração por letras sequenciais
dict_letters = {'' : '', 0 : '_A',1 : '_B', 2 : '_C', 3 : '_D', 4 : '_E',
                5 : '_F', 6 : '_G', 7 : '_H', 8 : '_I', 9 : '_J'}
pareceres_df['flag']= pareceres_df['flag'].map(dict_letters) 
pareceres_df.drop(['dupl'], axis=1, inplace=True) # remove a coluna dupl

print('\033[1m' + "\nFlags de contagem adicionadas.\n" + '\033[0m')

####### Criando a coluna com os arquivos no padrão Inova

# # Cria a coluna com o nome do arquivo no padão Inova
pareceres_df.loc[pareceres_df['download'] == False, 'folder'] = False # folder é false se não houver download
pareceres_df['inova_name'] = pareceres_df['nr_sici'] + pareceres_df['pi'] + '_' + pareceres_df['doc_name'] + '_' + data + pareceres_df['flag'] + '.pdf'
pareceres_df.loc[pareceres_df['download'] == False, 'inova_name'] = False # inova file name é false se não houver download
pareceres_df.drop(['flag', 'doc_name', 'nr_sici'], axis=1, inplace=True) # remove a coluna flag

# Os documetos com despacho 16.1, carta patente, não se encontram no servidor. Onde estão?

print('\033[1m' + "\nTudo pronto para começar o dowload! Iniciando.....\n" + '\033[0m')

##### Faz o download dos arquivos nas pastas

f_revista = "RPI" + revista + "_" + data
f_epat = "E_PAT_RPI" + revista + "_" + data

url_cam = "http://parecer.inpi.gov.br/download.php?cam=arquivos/RPI/" + revista + '/'

i = 1 # contador de posição de linha do dataframe
for folder,file_name,inova_name,download in zip(pareceres_df['folder'],pareceres_df['inpi_name'],pareceres_df['inova_name'],pareceres_df['download']):
    if download == True:
        f = f_revista + "/" + folder + "/" + f_epat
        if not os.path.exists(f):
            os.makedirs(f)
        url = url_cam + file_name
        arq_request = requests.get(url, verify = False)
        path = f + "/" + inova_name
        file = open(path, 'wb').write(arq_request.content) # faz o download do arquivo
        if file == 0: # se o arquivo não existir o documento baixado terá 0k
            path1 = path.split("/")[0] + "/" + path.split("/")[1] + "/" + path.split("/")[2] + "/" + path.split("/")[3]
            path2 = path.split("/")[0] + "/" + path.split("/")[1] + "/" + path.split("/")[2]
            path3 = path.split("/")[0] + "/" + path.split("/")[1]
            shutil.rmtree(path1)
            shutil.rmtree(path2)
            shutil.rmtree(path3)
            pareceres_df.at[i, 'download'] = "Erro! Verifique o site do INPI!"
        i+=1

print('\033[1m' + "\nDownload dos arquivos concluídos!\n" + '\033[0m')

########### Relatório de downloads

print('\033[1m' + "\nConstruindo relatório de downloads.....\n" + '\033[0m')

# excel com as informações de download
# cols = list(arquivos_df.columns.values) # list columns

pareceres_df = pareceres_df[['tecnologia', 'pi', 'despacho', 'inpi_name', 'inova_name', 'download', 'dupl_exig']] # muda a ordem das colunas
pareceres_df.loc[pareceres_df['download'] == False, 'download'] = "Arquivo inexistente" # se o arquivo é False desde o ínicio
pareceres_df.loc[pareceres_df['download'] == True, 'download'] = "Ok" # Se o statis True do arquivo não foi alterado
pareceres_df.loc[pareceres_df['despacho'] == "16.1", 'download'] = "Faça o donwload da carta patente no site INPI." # se o arquivo é False desde o ínicio

pareceres_df.loc[pareceres_df['dupl_exig'] == True, 'download'] = "Arquivos duplicados, verifique o site do INPI." # indica tecnologias com dois despachos na semana para verificação
pareceres_df.drop(['dupl_exig'], axis=1, inplace=True)

print("Relatório da semana concluído.")
# pareceres_df

# Gerar o relatório de downloads em excel
# nesse caso o revista é string, no caso de gestão é int
parecer_report(pareceres_df,f_revista + "/" + "RPI" + revista + ".xlsx",revista)


RPI 2586


Dados do SICI convertidos para um dataframe.



/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



Os arquivos estão disponíveis para download. Iniciando o processo.....



/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



Data da revista: 280720


Estrutura de pastas montada.


Coluna de busca defininda.


Dataframe do INPI preparado.


Dataframe dos pareceres criado.


Dataframes sici_df e finpi_df deletados.


Flags de contagem adicionadas.


Tudo pronto para começar o dowload! Iniciando.....



/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/frmartellini/miniconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'parecer.inpi.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



Download dos arquivos concluídos!


Construindo relatório de downloads.....

Relatório da semana concluído.


In [ ]:
#@title Criar o arquivo zip

# ## Para uso no Google Colab

# print('\033[1m' + "\nCompactando os arquivos para o download no computador.\n" + '\033[0m')

# # download dos arquivos em zip
# zip_file = f_revista + ".zip"
# zip_folder = f_revista

# # !zip -r /content/file.zip /content/Folder_To_Zip
# !zip -r "$zip_file" "$zip_folder"

# print('\033[1m' + "\nTudo terminado!\n" + '\033[0m')

# # Fazer o download dos arquivos para o computador
# # nem sempre funciona, melhor deixar manual
# files.download(zip_file)

In [ ]:
#@title Limpar arquivos da nuvem

# Para uso no Google Colab

# !rm "$arquivo"
# !rm -rf "$zip_folder"